In [1]:
import pandas as pd
df = pd.read_csv('C:/Users/reddy/eng_movie_subtitles.csv')

In [3]:
df.rename(columns={'Unnamed: 0': 'index'}, inplace=True)

In [4]:
df.head(5)

,index,name,file_content
0,0,The Message (1976),watch any video online with opensubtitles free...
1,1,Here Comes The Grump S01 E09 Joltin Jack In Bo...,ah theres princess dawn and terry with the 2 b...
2,2,Yumis Cells S02 E13 Episode 2 13 (2022),yumis cells 2 2 episode 36 extremely polite yu...
3,3,Yumis Cells S02 E14 Episode 2 14 (2022),watch any video online with opensubtitles free...
4,4,Broker (2022),1 watch any video online with opensubtitles fr...


In [5]:
from sentence_transformers import SentenceTransformer

def generate_embeddings(texts):
    model = SentenceTransformer('bert-base-nli-mean-tokens')
    embeddings = []
    for text_chunk in texts:
        chunk_embeddings = model.encode(text_chunk)
        embeddings.append(chunk_embeddings)
    return embeddings

def document_chunker(data, chunk_size=500, overlap_size=50):
    chunks = []
    start_idx = 0
    while start_idx < len(data):
        end_idx = min(start_idx + chunk_size, len(data))
        chunk = ' '.join(data[start_idx:end_idx])
        chunks.append(chunk)
        start_idx += chunk_size - overlap_size
    return chunks

In [6]:
texts = df['file_content'].tolist()

In [7]:
chunked_texts = document_chunker(texts)

In [8]:
chunk_embeddings = generate_embeddings(chunked_texts)

In [9]:
import chromadb

In [10]:
client = chromadb.PersistentClient(path="C:/search_engine/eng_subtitles")

In [11]:
collection = client.get_or_create_collection(name="eng_subtitles")

In [12]:
for index, row in df.iterrows():
    document_id = str(row['index'])  # Use a unique identifier for each document
    document_text = row['file_content']
    # document_embedding = corpus_embeddings[index % len(corpus_embeddings)]  # Assuming embeddings are aligned with the rows

    # ------------------------------------------------------------------------
    chunk_index = index % len(chunk_embeddings)
    document_embedding = chunk_embeddings[chunk_index]
    #--------------------------------------------------------------------------
    metadata = {'movie_name': row['name']}

    # Insert document into ChromaDB collection
    collection.add(ids=document_id, documents=[document_text], embeddings=[document_embedding.tolist()], metadatas=[metadata])


print("Insertion into ChromaDB collection complete")

Insertion into ChromaDB collection complete
